## Step 1: Install the dependencies scripts and python packages
In order to ran the whole program, you need:
```
    abc             # the compiled abc binary file
    abc.rc          # the alias of abc
    tautology.ipynb # this notebook
```

Besides, make sure your python version is above 3.4

In [3]:
import subprocess
import sys
import random
import os
import re
import matplotlib.pyplot as plt
from tqdm import tqdm
if not os.path.exists('abc'):
    raise EnvironmentError
if not os.path.exists('abc.rc'):
    raise EnvironmentError
if not os.path.exists('results'):
    os.makedirs('results')
# for Machine Learning
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

## Step 2: Define the function
Do not modify the function below

In [11]:
def print_tautology_blif(input_size, 
                         file = 'test.temp', 
                         shuffle_product = False,
                         shuffle_sum = False,
                         seq_sum = None):
    '''
    print_tautology_blif:
        1. Generate an tautology logic with <input_size> inputs
        2. Write the result to <file>
        3. Random Shufle the product if <shuffle_product> is True
        4. Random Shufle the sum if <shuffle_sum> is True
    '''
    with open(file,'w') as f:
        f.write('.model test\n')
        f.write('.inputs '+' '.join(['input'+str(index) for index in range(input_size)])+'\n')
        f.write('.outputs out\n')
        # single products in random order
        for combination in range(2**input_size):
            terms = [*range(input_size)]
            if shuffle_product:
                random.shuffle(terms)
            f.write('.names '+' '.join(['input'+str(index) for index in terms])+' node'+str(combination)+'\n')
            f.write(('{0:0'+str(input_size)+'b}').format(combination)+' 1\n')
        # sum all the products in random order
        if seq_sum is None:
            seq = [*range(2**input_size)]
            if shuffle_sum:
                random.shuffle(seq)
        else:
            seq = seq_sum
        f.write('.names '+' '.join(['node'+str(index) for index in seq])+' out\n')
        f.write('0'*(2**input_size)+' 0\n')
        f.write('.end\n')
        return seq
def run_synthesis(file, opt_command='compress2rs'):
    commands = [
        'read_blif {0}'.format(file),
        'strash',
        'ps',
        'write_blif src.temp',
        opt_command,
        'ps',
        'write_blif res.temp'
    ]
    shell_command = './abc -c \"{0}\"'.format(';'.join(commands))
    return subprocess.getoutput(shell_command)
def parse_report(report, verbose = False):
    assert report is not None
    result = None, None
    for report_line in report.split('\n'):
        obj = re.match(r'.*and = (.*) lev = (.*)', report_line, re.M|re.I)
        if obj is None:
            continue
        result = area, level = int(obj.group(1).strip()), int(obj.group(2).strip())
        if verbose:
            print('area = {0}, level = {1}'.format(area,level))
    return result
def clean_results():
    if os.path.exists('res.temp'):
        os.remove('res.temp')
    if os.path.exists('src.temp'):
        os.remove('src.temp')
    if os.path.exists('test.temp'):
        os.remove('test.temp')
def record_blif(bname):
    '''
    store the result of source code
        <bmark>_src.blif: the input
        <bmark>_abc.blif: the result after synthesis of abc
    '''
    subprocess.run('cat src.temp > results/{0}_src.blif'.format(bname), shell=True)
    subprocess.run('cat res.temp > results/{0}_abc.blif'.format(bname), shell=True)
def collect_data(episode=1000,
                 input_size=6,
                 shuffle_product=False,
                 shuffle_sum=True):
    '''
    run <episode> iterations and collect the information of seq, area
    '''
    number_episode = episode
    area_results = []
    data_set = []
    for _ in tqdm(range(number_episode), ncols = 80):
        # generate tautology
        seq = print_tautology_blif(input_size = input_size,
                            file = 'test.temp',
                            shuffle_product = shuffle_product,
                            shuffle_sum = shuffle_sum)
        # run synthesis
        report = run_synthesis(file = 'test.temp',
                            opt_command = 'compress2rs')
        # parse the report
        area, level = parse_report(report)

        if area > 0:
            record_blif('t5')
        area_results.append(area)
        data_set.append([seq, area])

        # clean the temp files
        clean_results()
    return data_set
def one_hot_encode(sequence, input_size):
    code = []
    for permutation in sequence:
        one_hot = [ int(_) for _ in ('{0:0'+str(input_size)+'b}').format(permutation) ]
        code = code + one_hot
    return code
def process_data(data, input_size, verbose=False):
    data_X = []
    data_Y = []
    for seq, area in data:
        data_X.append(one_hot_encode(seq, input_size))
        data_Y.append(int(area == 0))
    df = pd.DataFrame(data_X)
    df['Y'] = data_Y
    if verbose:
        print('Naive Guess = {0}/{1}'.format(sum(df.Y), len(df.Y)))
    return df
def train_test_split(dataframe, ratio=0.8):
    total_size = len(dataframe)
    train_size = round(total_size*ratio)
    test_size = total_size - train_size
    X_train = dataframe.iloc[:train_size,:-1]
    Y_train = dataframe.iloc[:train_size,-1]
    X_test = dataframe.iloc[train_size:,:-1]
    Y_test = dataframe.iloc[train_size:,-1]
    return X_train, X_test, Y_train, Y_test
def get_accuracy(Y_pred, Y_test, verbose=False):
    error = np.sum(abs(np.array(Y_pred)-np.array(Y_test)))
    accuracy = 1-error/float(len(Y_pred))
    if verbose:
        print('Accu. = {0:02f}%'.format(accuracy*100.0))
    return accuracy
def decision_tree_train(dataframe, ratio=0.9):
    '''
    Train the results stored in that dataframe with the train/test split ratio of <ratio>
    '''
    X_train, X_test, Y_train, Y_test = train_test_split(df, ratio=0.9)
    dtree = DecisionTreeClassifier()
    dtree.fit(X_train, Y_train)
    Y_pred = dtree.predict(X_test)
    return get_accuracy(Y_pred, Y_test, verbose=False)

## Step 3: Run the script  

In [5]:
data_set = collect_data(episode = 100,
                        input_size = 5,
                        shuffle_product = False,
                        shuffle_sum = True)
print('\n')
df = process_data(data_set, input_size = 6, verbose = True)
accuracy = decision_tree_train(df, ratio = 0.8)
print('Accu. = {0:02f}%'.format(accuracy*100.0))


100%|█████████████████████████████████████████| 100/100 [00:06<00:00, 15.44it/s]

Naive Guess = 100/100
Accu. = 100.000000%



In [6]:
# in case we find some unexpected results
for _ in range(len(df)):
    if df.Y[_] == 0:
        seq = data_set[_][0]
        print_tautology_blif(input_size=5,
                             seq_sum=seq)
        report = run_synthesis(file = 'test.temp',
                                    opt_command = 'compress2rs')
        area, level = parse_report(report)
        record_blif(bname='t5')
        clean_results()
        break


In [10]:
print_tautology_blif(input_size=10, shuffle_product=True, shuffle_sum=True)
clean_results()

In [12]:
print_tautology_blif(input_size=3, shuffle_product=True, shuffle_sum=True)


[0, 5, 1, 4, 6, 3, 2, 7]